<a href="https://colab.research.google.com/github/vanessajen/Sam-tracker-v4/blob/main/Tiktokshop_tracker.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# TikTok Shop T-Shirt Tracker v4 (Auto Rotate + Daily Compare)
# By Sam for Jenny 💛
# Tự động mỗi ngày: lưu 20 sản phẩm t-shirt → so sánh hôm sau → gửi báo cáo qua Telegram

import requests
from bs4 import BeautifulSoup
import pandas as pd
from datetime import datetime
import re
import os
import json

# === CONFIG ===
TRENDING_URL = "https://www.tiktok.com/business/creativecenter/inspiration/top-products/pc/en"
HEADERS = {"User-Agent": "Mozilla/5.0"}
HISTORY_FOLDER = "sales_history"
EXPORT_CSV_PATH = "tshirt_growth_report.csv"
MAX_PRODUCTS = 20

# === TELEGRAM CONFIG ===
TELEGRAM_BOT_TOKEN = "8118142644:AAEVkSRX8ACgeCsCS9h3IjC8Kraf_v_poPY"
TELEGRAM_CHAT_ID = "1157657879"

# === CREATE FOLDER ===
os.makedirs(HISTORY_FOLDER, exist_ok=True)
today_str = datetime.today().strftime('%Y-%m-%d')
yesterday_str = (datetime.today() - pd.Timedelta(days=1)).strftime('%Y-%m-%d')

today_path = f"{HISTORY_FOLDER}/{today_str}.json"
yesterday_path = f"{HISTORY_FOLDER}/{yesterday_str}.json"

# === SCRAPE FUNCTION ===
def scrape_trending_products():
    response = requests.get(TRENDING_URL, headers=HEADERS)
    soup = BeautifulSoup(response.text, 'html.parser')

    products = []
    for card in soup.select(".product-card"):
        name = card.select_one(".product-card-title")
        link = card.select_one("a")
        sales = card.find(text=re.compile("Sales"))

        if name and link and sales:
            name_text = name.text.strip()
            url = "https://www.tiktok.com" + link.get("href")
            sales_number = int(re.sub(r"[^0-9]", "", sales))

            products.append({
                "name": name_text,
                "url": url,
                "sales": sales_number
            })

    return products[:MAX_PRODUCTS]  # Giới hạn 20 sản phẩm mỗi ngày

# === LOAD DATA ===
today_data = scrape_trending_products()

# Save today's data
with open(today_path, "w") as f:
    json.dump(today_data, f, indent=2)

# Compare with yesterday if exists
results = []
if os.path.exists(yesterday_path):
    with open(yesterday_path) as f:
        yesterday_data = json.load(f)
        y_map = {item['name']: item for item in yesterday_data}

        for item in today_data:
            name = item['name']
            if name in y_map:
                y_sales = y_map[name]['sales']
                t_sales = item['sales']
                growth = (t_sales - y_sales) / y_sales if y_sales > 0 else 0
                if growth >= 0.07:
                    results.append({
                        "name": name,
                        "url": item['url'],
                        "sales_yesterday": y_sales,
                        "sales_today": t_sales,
                        "growth_rate_%": round(growth * 100, 2)
                    })

# === REPORT ===
if results:
    df = pd.DataFrame(results)
    print("\n📈 Danh sách sản phẩm T-Shirt tăng trưởng > 7% so với hôm qua:")
    print(df[['name', 'url', 'sales_yesterday', 'sales_today', 'growth_rate_%']])
    df.to_csv(EXPORT_CSV_PATH, index=False)
    print(f"\n📁 File kết quả đã lưu: {EXPORT_CSV_PATH}")

    def send_to_telegram(message):
        url = f"https://api.telegram.org/bot{TELEGRAM_BOT_TOKEN}/sendMessage"
        payload = {
            "chat_id": TELEGRAM_CHAT_ID,
            "text": message,
            "parse_mode": "HTML"
        }
        requests.post(url, data=payload)

    message = "\U0001F4CA <b>TikTokShop T-Shirt Growth Report</b>\n"
    for row in results:
        message += f"\n<b>{row['name']}</b>\n➡️ Growth: {row['growth_rate_%']}%\n🔗 <a href='{row['url']}'>View Product</a>\n"
    send_to_telegram(message)
    print("\n✉️ Đã gửi báo cáo qua Telegram!")
else:
    print("\n❌ Không có sản phẩm nào đạt mức tăng trưởng yêu cầu hôm nay.")
